SI BESOIN ANALYSE TWEETS 16M RAWS HUGGING FACE

In [85]:
from datasets import load_dataset

ds = load_dataset("ckandemir/bitcoin_tweets_sentiment_kaggle")

In [86]:
small = ds["train"].select(range(1000))  # 1000 premières lignes
df_small = small.to_pandas()
df_small.head()


,Date,text,Sentiment
0,2019-06-12,Are you interested in Bitcoin mining for FREE!...,Positive
1,2019-07-08,"QA Analyst - IT Services Company ( WESTBURY, U...",Positive
2,2019-05-22,🌍22/05/2019🌍\nA Abakus a qual a NovaDAX perten...,Negative
3,2019-05-16,@Quark_Chain #QKC #Blockchain #BTC #ETH #block...,Negative
4,2018-07-21,"July 21, 2018 04:00 AM EDT\nLast 4 hours, BTC ...",Positive


NETTOYAGE DES DATASETS

In [87]:
import pandas as pd
import numpy as np

from pathlib import Path


chargement des datasets

In [88]:
# Chemins
BASE_PATH = Path("../data/brutes/sentiment")

fear_greed_path = BASE_PATH / "fear_and_greed_index.csv"
news_sentiment_path = BASE_PATH / "bitcoin-news-sentiments.csv"
news_sentiment_21_24_path = BASE_PATH / "bitcoin_sentiments_21_24.csv"

# Load
df_fear = pd.read_csv(fear_greed_path)
df_news = pd.read_csv(news_sentiment_path)
df_news_21_24 = pd.read_csv(news_sentiment_21_24_path)

# Quick check
print(df_fear.head())
print(df_news.head())
print(df_news_21_24.head())


         date  fng_value fng_classification
0  31-12-2025       21.0       Extreme Fear
1  30-12-2025       23.0       Extreme Fear
2  29-12-2025       24.0       Extreme Fear
3  28-12-2025       24.0       Extreme Fear
4  27-12-2025       23.0       Extreme Fear
                                           Headlines        Date Sentiments  \
0  Price analysis 3/15: BTC, ETH, BNB, SOL, XRP, ...  2024-03-15    neutral   
1  Bitcoin drops 9% from its ATH as the market sh...  2024-03-15   negative   
2  UK court rules Craig Wright not Nakamoto, endi...  2024-03-15    neutral   
3  Wealth manager Cetera adds spot BTC ETFs to it...  2024-03-15    neutral   
4  Who is ‘Mr. 100’? Mysterious Bitcoin whale bec...  2024-03-15    neutral   

                                   Source  
0  https://cointelegraph.com/tags/bitcoin  
1  https://cointelegraph.com/tags/bitcoin  
2  https://cointelegraph.com/tags/bitcoin  
3  https://cointelegraph.com/tags/bitcoin  
4  https://cointelegraph.com/tags/bitcoin

Fear Greed

timestamps (UTC) + Nettoyage

In [89]:
bad_rows = df_fear[pd.to_datetime(
    df_fear["date"], 
    format="%d-%m-%Y", 
    errors="coerce"
).isna()]

bad_rows.head()


,date,fng_value,fng_classification
2887,\t],NaN,NaN
2888,"\t""metadata"": {",NaN,NaN
2889,"\t\t""error"": null",NaN,NaN
2890,\t},NaN,NaN
2891,},NaN,NaN


In [90]:
df_fear = pd.read_csv("../data/brutes/sentiment/fear_and_greed_index.csv")

# Nettoyage des strings
df_fear["date"] = df_fear["date"].astype(str).str.strip()

# Conversion robuste
df_fear["date"] = pd.to_datetime(
    df_fear["date"],
    format="%d-%m-%Y",
    utc=True,
    errors="coerce"   # 🔥 clé ici
)

# Supprimer les lignes invalides
df_fear = df_fear.dropna(subset=["date"])

# Renommer / garder colonnes utiles
df_fear = df_fear.rename(columns={"fng_value": "fear_greed_index"})
df_fear = df_fear[["date", "fear_greed_index"]]

df_fear = df_fear.sort_values("date")


Mise sous 4h pour ensuite coller au btc

In [91]:
df_fear_4h = (
    df_fear
    .set_index("date")
    .resample("4H")
    .ffill()
    .reset_index()
)

df_fear_4h.head(10)


/var/folders/ps/d73lxnw91lq7j2cs28b1z6pw0000gn/T/ipykernel_34223/1172527637.py:4: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  .resample("4H")


,date,fear_greed_index
0,2018-02-01 00:00:00+00:00,30.0
1,2018-02-01 04:00:00+00:00,30.0
2,2018-02-01 08:00:00+00:00,30.0
3,2018-02-01 12:00:00+00:00,30.0
4,2018-02-01 16:00:00+00:00,30.0
5,2018-02-01 20:00:00+00:00,30.0
6,2018-02-02 00:00:00+00:00,15.0
7,2018-02-02 04:00:00+00:00,15.0
8,2018-02-02 08:00:00+00:00,15.0
9,2018-02-02 12:00:00+00:00,15.0


In [92]:
df_fear_4h.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17341 entries, 0 to 17340
Data columns (total 2 columns):
 #   Column            Non-Null Count  Dtype              
---  ------            --------------  -----              
 0   date              17341 non-null  datetime64[ns, UTC]
 1   fear_greed_index  17341 non-null  float64            
dtypes: datetime64[ns, UTC](1), float64(1)
memory usage: 271.1 KB


Nettoyage FinGpt

In [93]:
print(df_news.head())

                                           Headlines        Date Sentiments  \
0  Price analysis 3/15: BTC, ETH, BNB, SOL, XRP, ...  2024-03-15    neutral   
1  Bitcoin drops 9% from its ATH as the market sh...  2024-03-15   negative   
2  UK court rules Craig Wright not Nakamoto, endi...  2024-03-15    neutral   
3  Wealth manager Cetera adds spot BTC ETFs to it...  2024-03-15    neutral   
4  Who is ‘Mr. 100’? Mysterious Bitcoin whale bec...  2024-03-15    neutral   

                                   Source  
0  https://cointelegraph.com/tags/bitcoin  
1  https://cointelegraph.com/tags/bitcoin  
2  https://cointelegraph.com/tags/bitcoin  
3  https://cointelegraph.com/tags/bitcoin  
4  https://cointelegraph.com/tags/bitcoin  


In [94]:
# Convert date to datetime UTC
df_news["date"] = pd.to_datetime(df_news["Date"], utc=True)

# Mapping sentiment texte → numérique
sentiment_map = {
    "positive": 1,
    "neutral": 0,
    "negative": -1
}

df_news["sentiment_num"] = df_news["Sentiments"].str.lower().map(sentiment_map)

# Drop rows with unknown sentiment
df_news = df_news.dropna(subset=["sentiment_num"])

# Keep only useful columns
df_news = df_news[["date", "sentiment_num"]]

# Sort
df_news = df_news.sort_values("date").reset_index(drop=True)

df_news.head()

,date,sentiment_num
0,2011-06-16 00:00:00+00:00,0
1,2012-02-17 00:00:00+00:00,-1
2,2013-01-03 00:00:00+00:00,0
3,2013-01-27 00:00:00+00:00,-1
4,2013-03-20 00:00:00+00:00,-1


In [95]:
df_news_4h = (
    df_news
    .set_index("date")
    .resample("4H")
    .agg(
        news_sentiment_mean=("sentiment_num", "mean"),
        news_sentiment_std=("sentiment_num", "std"),
        news_count=("sentiment_num", "count")
    )
    .reset_index()
)

df_news_4h.head()


/var/folders/ps/d73lxnw91lq7j2cs28b1z6pw0000gn/T/ipykernel_34223/3033012645.py:4: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  .resample("4H")


,date,news_sentiment_mean,news_sentiment_std,news_count
0,2011-06-16 00:00:00+00:00,0.0,NaN,1
1,2011-06-16 04:00:00+00:00,NaN,NaN,0
2,2011-06-16 08:00:00+00:00,NaN,NaN,0
3,2011-06-16 12:00:00+00:00,NaN,NaN,0
4,2011-06-16 16:00:00+00:00,NaN,NaN,0


In [96]:
df_news_4h.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27937 entries, 0 to 27936
Data columns (total 4 columns):
 #   Column               Non-Null Count  Dtype              
---  ------               --------------  -----              
 0   date                 27937 non-null  datetime64[ns, UTC]
 1   news_sentiment_mean  3767 non-null   float64            
 2   news_sentiment_std   3527 non-null   float64            
 3   news_count           27937 non-null  int64              
dtypes: datetime64[ns, UTC](1), float64(2), int64(1)
memory usage: 873.2 KB


NETTOYAGE SENTIMENT_21_24

In [97]:
#sentiment_21_24
print(df_news_21_24.head())

                  date                                  Short Description  \
0  2021-11-05 04:42:00  Bitcoin price is consolidating near the USD 62...   
1  2021-11-05 08:15:00  Congress could finally approve or reject the m...   
2  2021-11-05 10:24:00  Bitcoin increasingly becoming a political inst...   
3  2021-11-05 16:58:00  There is still potential for the price of bitc...   
4  2021-11-05 21:00:00  'Several companies' are looking to Latin Ameri...   

   Accurate Sentiments  
0             0.998558  
1             0.000000  
2             0.000000  
3             0.999458  
4             0.000000  


In [98]:
# Convert date
df_news_21_24["date"] = pd.to_datetime(df_news_21_24["date"], utc=True)

# Si sentiment ∈ [0,1], recentrer sur [-1,1]
df_news_21_24["Accurate Sentiments"] = df_news_21_24["Accurate Sentiments"] * 2 - 1

# Sort
df_news_21_24 = df_news_21_24.sort_values("date")

In [99]:
df_news_21_24_4h = (
    df_news_21_24
    .set_index("date")
    .resample("4H")
    .agg(
        news21_24_sentiment_mean=("Accurate Sentiments", "mean"),
        news21_24_sentiment_std=("Accurate Sentiments", "std"),
        news21_24_count=("Accurate Sentiments", "count")
    )
    .reset_index()
)

df_news_21_24_4h.head()


/var/folders/ps/d73lxnw91lq7j2cs28b1z6pw0000gn/T/ipykernel_34223/1205578033.py:4: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  .resample("4H")


,date,news21_24_sentiment_mean,news21_24_sentiment_std,news21_24_count
0,2021-11-05 04:00:00+00:00,0.997116,NaN,1
1,2021-11-05 08:00:00+00:00,-1.000000,0.0,2
2,2021-11-05 12:00:00+00:00,NaN,NaN,0
3,2021-11-05 16:00:00+00:00,0.998915,NaN,1
4,2021-11-05 20:00:00+00:00,-1.000000,NaN,1


FUSION

In [100]:
# Debug: print column names to ensure consistent 'date' column before merging
print('df_fear_4h columns:', df_fear_4h.columns.tolist())
print('df_news_4h columns:', df_news_4h.columns.tolist())
print('df_news_21_24_4h columns:', df_news_21_24_4h.columns.tolist())

# Merge on the common 'date' column (all frames now have a lowercase 'date')
df_sentiment = df_fear_4h.merge(
    df_news_4h, on="date", how="left"
).merge(
    df_news_21_24_4h, on="date", how="left"
)
df_sentiment.head()


df_fear_4h columns: ['date', 'fear_greed_index']
df_news_4h columns: ['date', 'news_sentiment_mean', 'news_sentiment_std', 'news_count']
df_news_21_24_4h columns: ['date', 'news21_24_sentiment_mean', 'news21_24_sentiment_std', 'news21_24_count']


,date,fear_greed_index,news_sentiment_mean,news_sentiment_std,news_count,news21_24_sentiment_mean,news21_24_sentiment_std,news21_24_count
0,2018-02-01 00:00:00+00:00,30.0,-0.47619,0.749603,21.0,NaN,NaN,NaN
1,2018-02-01 04:00:00+00:00,30.0,NaN,NaN,0.0,NaN,NaN,NaN
2,2018-02-01 08:00:00+00:00,30.0,NaN,NaN,0.0,NaN,NaN,NaN
3,2018-02-01 12:00:00+00:00,30.0,NaN,NaN,0.0,NaN,NaN,NaN
4,2018-02-01 16:00:00+00:00,30.0,NaN,NaN,0.0,NaN,NaN,NaN


GESTION DES NaN

In [101]:
df_sentiment["news_sentiment_mean"] = df_sentiment["news_sentiment_mean"].fillna(0)
df_sentiment["news_sentiment_std"] = df_sentiment["news_sentiment_std"].fillna(0)
df_sentiment["news_count"] = df_sentiment["news_count"].fillna(0)

df_sentiment["news21_24_sentiment_mean"] = df_sentiment["news21_24_sentiment_mean"].fillna(0)
df_sentiment["news21_24_sentiment_std"] = df_sentiment["news21_24_sentiment_std"].fillna(0)
df_sentiment["news21_24_count"] = df_sentiment["news21_24_count"].fillna(0)

ajout d'un indicateur pour que le  modèle puisse distinguer :0 car neutre, 0 car aucune info

In [105]:
df_sentiment["has_news"] = ((df_sentiment["news_count"] > 0) | 
                            (df_sentiment["news21_24_count"] > 0)
                            ).astype(int)


In [103]:
df_sentiment.info()
df_sentiment.describe()
df_sentiment.head(5)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17341 entries, 0 to 17340
Data columns (total 9 columns):
 #   Column                    Non-Null Count  Dtype              
---  ------                    --------------  -----              
 0   date                      17341 non-null  datetime64[ns, UTC]
 1   fear_greed_index          17341 non-null  float64            
 2   news_sentiment_mean       17341 non-null  float64            
 3   news_sentiment_std        17341 non-null  float64            
 4   news_count                17341 non-null  float64            
 5   news21_24_sentiment_mean  17341 non-null  float64            
 6   news21_24_sentiment_std   17341 non-null  float64            
 7   news21_24_count           17341 non-null  float64            
 8   has_news                  17341 non-null  int64              
dtypes: datetime64[ns, UTC](1), float64(7), int64(1)
memory usage: 1.2 MB


,date,fear_greed_index,news_sentiment_mean,news_sentiment_std,news_count,news21_24_sentiment_mean,news21_24_sentiment_std,news21_24_count,has_news
0,2018-02-01 00:00:00+00:00,30.0,-0.47619,0.749603,21.0,0.0,0.0,0.0,1
1,2018-02-01 04:00:00+00:00,30.0,0.00000,0.000000,0.0,0.0,0.0,0.0,0
2,2018-02-01 08:00:00+00:00,30.0,0.00000,0.000000,0.0,0.0,0.0,0.0,0
3,2018-02-01 12:00:00+00:00,30.0,0.00000,0.000000,0.0,0.0,0.0,0.0,0
4,2018-02-01 16:00:00+00:00,30.0,0.00000,0.000000,0.0,0.0,0.0,0.0,0


CREATION CSV MERGER

In [106]:
OUTPUT_PATH = Path("../data/nettoyé")
OUTPUT_PATH.mkdir(exist_ok=True)

df_sentiment.to_csv(OUTPUT_PATH / "sentiment_4h.csv", index=False)
